# Satifsying requirements

In [ ]:
!pip install pennylane --upgrade

In [ ]:
!pip install torch

In [39]:
import torch
import pennylane as qml
import numpy as np

%matplotlib inline

In [102]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


# Downloading data

In [103]:
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

# Preparing data with DataLoaders

In [104]:
from torch.utils.data import DataLoader

loaders = {
    'train' : torch.utils.data.DataLoader(train_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1,
                                          pin_memory=True),
    
    'test'  : torch.utils.data.DataLoader(test_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1,
                                          pin_memory=True),
}

# Defining a NN

In [105]:
n_qubits = 5
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(features=inputs, wires=range(n_qubits))
    
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    
    return [qml.expval(qml.PauliY(wires=i)) for i in range(n_qubits)]

In [106]:
n_layers = 3
weight_shapes = {"weights": (n_layers, n_qubits, 3)}

In [107]:
import torch.nn as nn

class HybridNN(nn.Module):
    def __init__(self):
        super(HybridNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(
                in_channels=16,              
                out_channels=32,            
                kernel_size=5,              
                stride=1,                   
                padding=2,    
            ),     
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),                
        )
        self.fc_1 = nn.Linear(32 * 7 * 7, 20)
        
        # LIST USAGE?
        self.qlayer_1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.qlayer_2 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.qlayer_3 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.qlayer_4 = qml.qnn.TorchLayer(qnode, weight_shapes)
        
        self.qlayer_1.to(device)
        self.qlayer_2.to(device)
        self.qlayer_3.to(device)
        self.qlayer_4.to(device)
        
        self.fc_2 = nn.Linear(20, 10)

        
    def forward(self, x):

        x = self.conv1(x)

        x = self.conv2(x)

        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1) 

        x = self.fc_1(x)
        #print('Before split')
        x_1, x_2, x_3, x_4 = torch.split(x, 5, dim=1) # second argument is number of elements in one new tensor
        #print('After split')
        #x = torch.Tensor(0)
        
        x_1 = self.qlayer_1(x_1)
        x_2 = self.qlayer_2(x_2)
        x_3 = self.qlayer_3(x_3)
        x_4 = self.qlayer_4(x_4)
        
        #print(x.device)
        
        x = torch.cat([x_1, x_2, x_3, x_4], axis=1)
        x = x.to(device)
        
        logits = self.fc_2(x)
        
        return logits

In [108]:
hnn = HybridNN()
hnn = hnn.to(device)
print(hnn)

HybridNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_1): Linear(in_features=1568, out_features=20, bias=True)
  (qlayer_1): <Quantum Torch Layer: func=qnode>
  (qlayer_2): <Quantum Torch Layer: func=qnode>
  (qlayer_3): <Quantum Torch Layer: func=qnode>
  (qlayer_4): <Quantum Torch Layer: func=qnode>
  (fc_2): Linear(in_features=20, out_features=10, bias=True)
)


# Training

In [109]:
loss_func = nn.CrossEntropyLoss()

In [110]:
from torch import optim

optimizer = optim.Adam(hnn.parameters(), lr = 0.01)  

In [111]:
from tqdm.notebook import trange
from torch.autograd import Variable

def train(num_epochs, model, loaders):
    
    model.train()
        
    # Train the model
    total_step = len(loaders['train'])
        
    for epoch in trange(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            
            b_x, b_y = images.to(device), labels.to(device)

            output = model(b_x)             
            loss = loss_func(output, b_y)
            
            optimizer.zero_grad()           
            
            loss.backward()               
            optimizer.step()                
            
            if (i+1) % 10 >= 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
        print('\n')

In [ ]:
num_epochs = 10

train(num_epochs, hnn, loaders)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch [1/10], Step [1/600], Loss: 2.3107
Epoch [1/10], Step [2/600], Loss: 2.3104
Epoch [1/10], Step [3/600], Loss: 2.2895
Epoch [1/10], Step [4/600], Loss: 2.2878
Epoch [1/10], Step [5/600], Loss: 2.2382
Epoch [1/10], Step [6/600], Loss: 2.2370
Epoch [1/10], Step [7/600], Loss: 2.1404


In [ ]:
torch.save(hnn.state_dict(), 'trained_model_10qubits.pt')